In [1]:
from pathlib import Path
from dotenv import load_dotenv
import xlrd
import os
from sqlalchemy.engine import create_engine
from sqlalchemy.exc import SQLAlchemyError
import cx_Oracle
import pandas as pd
env_path = Path('C:/Users/xxx/xxx/Location') / 'credentials/pp.env'
load_dotenv(dotenv_path=env_path)

from jinjasql import JinjaSql
j = JinjaSql(param_style='pyformat')

In [2]:
def get_oracle_conn():
    DIALECT = 'oracle'
    SQL_DRIVER = 'cx_oracle'
    USERNAME = os.environ.get('QSP_ORACLE_USER') #enter your username
    PASSWORD = os.environ.get('QSP_ORACLE_PWD') #enter your password
    HOST = os.environ.get('QSP_ORACLE_HOST') #enter the oracle db host url
    PORT = os.environ.get('QSP_ORACLE_PORT') # enter the oracle port number
    SERVICE = os.environ.get('QSP_ORACLE_SERVICE') # enter the oracle db service name
    ENGINE_PATH_WIN_AUTH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE
    
    return ENGINE_PATH_WIN_AUTH, cx_Oracle.connect(user=USERNAME, password=PASSWORD,dsn=f"{HOST}/{SERVICE}")

In [4]:
#conn_string, connection = get_oracle_conn()
#print(os.environ.get('QSP_ORACLE_USER'))
#print(os.environ.get('QSP_ORACLE_PWD'))
#print(os.environ.get('QSP_ORACLE_HOST'))
#print(os.environ.get('QSP_ORACLE_PORT'))
#print(os.environ.get('QSP_ORACLE_SERVICE'))
#print(conn_string)
#print(connection)

In [5]:
conn_string, connection = get_oracle_conn()

In [51]:
def get_excel_values(file_name, path=Path('C:/Users/xxx/xxx/Location/')):
    df = pd.read_excel(path / file_name, sheet_name=0)   
    return df

In [52]:
data = get_excel_values('UPDATE_parm_geo_planet.xlsx')

In [53]:
data.shape

(12538, 3)

In [54]:
data.head(2)

,ID,CURRENT_NAME,NEW_NAME
0,12624108,Gemeinde Murrhardt,Murrhardt
1,12623114,Gemeinde Hemau,Hemau


In [55]:
print(data[data['ID']==12623747])

           ID           CURRENT_NAME      NEW_NAME
100  12623747  Gemeinde Eberhardzell  Eberhardzell


In [56]:
filtered_data = data[data['ID'] ==12623747]

In [57]:
filtered_data.shape


(1, 3)

In [58]:
data = get_excel_values('UPDATE_parm_geo_planet.xlsx')
filtered_data = data[data['ID'] ==12623747]

In [59]:
conn_string, connection = get_oracle_conn()

In [46]:
for index, row in filtered_data.iterrows():
    params = {
        'id': row['ID'], 'new_name':row['NEW_NAME']   
          }  
    update_geo_planet = '''update AIDATAUSER.PARM_GEO_PLANET set name = '{{new_name}}' where id = '{{id}}'
      '''
    query, bind_params = j.prepare_query(update_geo_planet, params)
    sql_update_parm_geo_planet = query % bind_params
    print(sql_update_parm_geo_planet)
        
  
    script_get_comments = '''select '{"objBefore":"{\"id\":\"'||pgp.id||'\",\"name\":\"'||pgp.name||'\",\"iso\":\"'||pgp.iso||'\",\"parent_id\":\"'||pgp.parentid||'\",\"place_type\":\"'||pgp.placetype||'\",\"provice_id\":\"'||pgp.provinceid||
           '\",\"county_id\":\"'||pgp.countyid||'\",\"country_id\":\"'||pgp.countryid||'\",\"continent_id\":\"'||pgp.continentid||'\",\"parent_name\":\"'||parnt.name||'\",\"province_name\":\"'||province.name||
           '\",\"country_name\":\"'||country.name||'\",\"continent_name\":\"'||continent.name||'"},"objAfter":"{\"id\":\"'||pgp.id||'\",\"name\":\"{{new_name}}\",\"iso\":\"'||pgp.iso||'\",\"parent_id\":\"'||pgp.parentid||'\",\"place_type\":\"'||pgp.placetype||'\",\"provice_id\":\"'||pgp.provinceid|| 
           '\",\"county_id\":\"'||pgp.countyid||'\",\"country_id\":\"'||pgp.countryid||'\",\"continent_id\":\"'||pgp.continentid||'\",\"parent_name\":\"'||parnt.name||'\",\"province_name\":\"'||province.name||
           '\",\"country_name\":\"'||country.name||'\",\"continent_name\":\"'||continent.name||'"}"}'  as comments      
           from aidatauser.parm_geo_planet pgp
        left join aidatauser.parm_geo_planet parnt on pgp.parentid = parnt.id
        left join aidatauser.parm_geo_planet province on pgp.provinceid = province.id
        left join aidatauser.parm_geo_planet country on pgp.countryid = country.id
        left join aidatauser.parm_geo_planet continent on pgp.continentid = continent.id
        where pgp.id = {{id}}
        '''
    query, bind_params = j.prepare_query(script_get_comments, params)
    #print(query % bind_params)
    
    sql_get_comments = query % bind_params
    
    cur = connection.cursor()
    cur.execute(sql_get_comments)
    ans = cur.fetchall()
    for comments in ans:
        #print(comments[0])
        v_comment = comments[0]
    
    params_comment = {
        'id': row['ID'],
        'comment': v_comment  
          } 
    
    insert_general_history = '''INSERT INTO AIDATAUSER.GENERAL_HISTORY  (SOURCE_ID,HISTORY_TYPE,HISTORY_WHEN,HISTORY_WHO,HISTORY_ACTION,COMMENTS,CREATE_TIME,UPDATE_TIME,ID,OBJECT_DATA,ACTION_SOURCE,ACTOR_ID,ACTOR_TYPE) 
                     VALUES ({{id}},'GEO_UPDATE',systimestamp,'jitesh.pillai','Update location','{{comment}}',systimestamp,systimestamp,NULL,NULL,NULL,NULL,NULL)
    '''
    query, bind_params = j.prepare_query(insert_general_history, params_comment)
    sql_insert_general_history = query % bind_params
    print(sql_insert_general_history) 
   
    ## Execute final scripts update parm_geo_planet and insert general_history then apply commit
    #print(sql_update_parm_geo_planet)
    cursor = connection.cursor()    
    cursor.execute(sql_update_parm_geo_planet)
    cursor.execute(sql_insert_general_history)
    ans = cur.fetchall()
    connection.commit()
    cursor.close()
    

update AIDATAUSER.PARM_GEO_PLANET set name = 'Eberhardzell' where id = '12623747'
      
INSERT INTO AIDATAUSER.GENERAL_HISTORY  (SOURCE_ID,HISTORY_TYPE,HISTORY_WHEN,HISTORY_WHO,HISTORY_ACTION,COMMENTS,CREATE_TIME,UPDATE_TIME,ID,OBJECT_DATA,ACTION_SOURCE,ACTOR_ID,ACTOR_TYPE) 
                     VALUES (12623747,'GEO_UPDATE',systimestamp,'jitesh.pillai','Update location','{"objBefore":"{"id":"12623747","name":"Gemeinde Eberhardzell","iso":"DE","parent_id":"12596779","place_type":"City","provice_id":"2345481","county_id":"12596779","country_id":"23424829","continent_id":"24865675","parent_name":"Landkreis Biberach","province_name":"Baden-Wurttemberg","country_name":"Germany","continent_name":"Europe"},"objAfter":"{"id":"12623747","name":"Eberhardzell","iso":"DE","parent_id":"12596779","place_type":"City","provice_id":"2345481","county_id":"12596779","country_id":"23424829","continent_id":"24865675","parent_name":"Landkreis Biberach","province_name":"Baden-Wurttemberg","country_name":"G